taz control total check script

8.3.2
- use raw control total
9.0.0
- mining
    - raw control total
everything else:
- raw control total + nonremm taz

Notes about mining control total and non remm taz
- some redundancy when adding mining jobs to 9.0
- mining is the only sector where the remm control equals the tdm control (salt lake)
- for taz 9.0, mining jobs in kennecott mine are assigned by tdm template, adjusted using county controls (?),then overwritten by nonremm taz
- for taz 8.3.2, mining jobs in kennecott mine are assigned by tdm template, adjusted using county controls (?)
- only one employment controls file shared by both 
- tdm_template file for each taz vintage
- non remm taz file used only by the taz 9.0
- controls totals are slightly different in 8.3.2 taz than 9.0 taz due to non remm tazz processing

In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [ ]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [2]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
employment_controls = pd.read_csv(r"E:\Projects\REMM2_For_Python3_Internal_Use\data\employment_controls.csv")
nonremm_taz = pd.read_csv(r"E:\Projects\REMM2_For_Python3_Internal_Use\data\WF NonREMM TAZ Forecast - 2022-12-20.csv")
year = 2019

In [17]:
# ec11 = employment_controls[(employment_controls['year']== year) & (employment_controls['cid'] == 11)].copy()
# ec35 = employment_controls[(employment_controls['year']== year) & (employment_controls['cid'] == 35)].copy()
# ec49 = employment_controls[(employment_controls['year']== year) & (employment_controls['cid'] == 49)].copy()
# ec57 = employment_controls[(employment_controls['year']== year) & (employment_controls['cid'] == 57)].copy()

In [20]:
ec_year = employment_controls[(employment_controls['year']== year)].copy()
employment_by_county_832 = ec_year.pivot(index='cid', columns='sector_id', values='number_of_jobs').reset_index()
employment_by_county_832 # compare taz 8.3.2 to this

sector_id,cid,1,2,3,4,5,6,7,8,9,10,11,12
0,11,10863,13618,40714,15859,14769,18991,46724,150,20064,10730,957,4434
1,35,55502,52051,140749,73790,59851,118799,247701,3526,85778,84870,1211,25831
2,49,20690,30193,58372,32847,21610,54353,85239,667,42926,15416,3462,10786
3,57,8250,8672,23865,14497,15224,9083,34562,143,15279,8189,1482,3236


In [28]:
nrt_year = nonremm_taz[nonremm_taz['YEAR']==year][['CO_FIPS', 'RETL','FOOD','MANU',	'WSLE',	'OFFI',	'GVED',	'HLTH',	'OTHR',	'AGRI',	'MING',	'CONS',	'HBJ']].copy()
nrt_year_county_sum = nrt_year.groupby('CO_FIPS')[['RETL','FOOD','MANU','WSLE',	'OFFI',	'GVED',	'HLTH',	'OTHR',	'AGRI',	'MING',	'CONS',	'HBJ']].sum().reset_index()
nrt_year_county_sum

,CO_FIPS,RETL,FOOD,MANU,WSLE,OFFI,GVED,HLTH,OTHR,AGRI,MING,CONS,HBJ
0,11,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0
1,35,70.8,704.1,0,0,8.5,93.3,0.0,319.4,0,1554,10,0
2,49,121.7,150.0,0,0,0.8,20.5,21.0,181.0,0,0,10,0
3,57,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0


In [34]:
ec_nrt = employment_by_county_832.merge(nrt_year_county_sum, left_on='cid', right_on='CO_FIPS', how='left')
ec_nrt['RETL'] = ec_nrt['RETL'] + ec_nrt[9]
ec_nrt['FOOD'] = ec_nrt['FOOD'] + ec_nrt[1]
ec_nrt['MANU'] = ec_nrt['MANU'] + ec_nrt[5]
ec_nrt['WSLE'] = ec_nrt['WSLE'] + ec_nrt[10]
ec_nrt['OFFI'] = ec_nrt['OFFI'] + ec_nrt[6]
ec_nrt['GVED'] = ec_nrt['GVED'] + ec_nrt[3]
ec_nrt['HLTH'] = ec_nrt['HLTH'] + ec_nrt[4]
ec_nrt['OTHR'] = ec_nrt['OTHR'] + ec_nrt[7]
ec_nrt['AGRI'] = ec_nrt['AGRI'] + ec_nrt[11]
ec_nrt['MING'] = ec_nrt[8]
ec_nrt['CONS'] = ec_nrt['CONS'] + ec_nrt[2]
ec_nrt['HBJ'] = ec_nrt['HBJ'] + ec_nrt[12]
employment_by_county_900 = ec_nrt[['CO_FIPS', 'RETL','FOOD','MANU',	'WSLE',	'OFFI',	'GVED',	'HLTH',	'OTHR',	'AGRI',	'MING',	'CONS',	'HBJ']].copy()
employment_by_county_900 # compare to taz 9.0.0

,CO_FIPS,RETL,FOOD,MANU,WSLE,OFFI,GVED,HLTH,OTHR,AGRI,MING,CONS,HBJ
0,11,20064.0,10863.0,14769,10730,18991.0,40714.0,15859.0,46724.0,957,150,13618,4434
1,35,85848.8,56206.1,59851,84870,118807.5,140842.3,73790.0,248020.4,1211,3526,52061,25831
2,49,43047.7,20840.0,21610,15416,54353.8,58392.5,32868.0,85420.0,3462,667,30203,10786
3,57,15279.0,8250.0,15224,8189,9083.0,23865.0,14497.0,34562.0,1482,143,8672,3236


In [ ]:
# get SE File for year and compare
modelqueen = False

se_832_folder = r"E:\Projects\REMM2_For_Python3_Internal_Use\TDM\_TDMv8.3.2_REMM\1_Inputs\2_SEData\REMM"
se_900_folder = r"E:\Projects\REMM2_For_Python3_Internal_Use\TDM\_TDMv8.3.2_REMM\1_Inputs\2_SEData\REMM"
se_folder = r"E:\Projects\REMM2_For_Python3_Internal_Use\TDM\_TDMv8.3.2_REMM\1_Inputs\2_SEData\REMM"